In [1]:
import warnings
warnings.simplefilter('ignore')

import logging
logging.basicConfig(filename='GRESKE.log')

# Scientific libraries
import numpy as np
import scipy.integrate as integrate

# Import Pandas
import pandas as pd

# Astro
import astropy.io.fits as fits
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo

# Graphic libraries
# if script:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

#if notebook:
#%matplotlib notebook
#import matplotlib.pyplot as plt


#optional 3ML imports

from threeML import *
from threeML.io.file_utils import *
from threeML.random_variates import RandomVariates

import os
import re

# my new imports
from grb_list import besties_list, multies_list, pulses_list, names_list
from fit_results_getter import ResultGetter, ResultGetter_single, ResultGetter_pulse
from models_params import Variates, Params
from writers import models_writer, write_grb_data
from betas_popper import evil_betas_writer, Beta_violator
from physical_properties import my_band, my_cutoffpl, get_betas, Epeak_rest_frame, Flux_and_lumin_rest_frame

In [2]:
besties = besties_list()
multies = multies_list()
pulsies = pulses_list()
grb_names = names_list()

In [3]:
rootdir = '/data29s/fermi/abacelj/GRB_correlations/GRB_builder/'

### main part

In [4]:
# get all timebins results, new way; each result is list of timebins of that grb

fit_results = []

for trig, GRB_name, z in besties:
    pulse = 0    
    fit_result = ResultGetter_pulse(GRB_name, pulse)
    fit_results.append(fit_result)

for trig, GRB_name, z in multies:
    
    with within_directory(rootdir):

        with within_directory(GRB_name):
            number_of_pulses = 0
            for i in range(1,10):
                pulse_dir = 'prepared_pha_files_%d'%i
                if(path_exists_and_is_directory(pulse_dir)==True):
                    number_of_pulses += 1

        for pulse in range(1,number_of_pulses+1):
            
            fit_result = ResultGetter_pulse(GRB_name, pulse)
            fit_results.append(fit_result)

In [5]:
len(fit_results)

38

In [6]:
peak_bins_list = pd.read_csv('peak_bins', delimiter='\t')

In [7]:
peak_bins_id = peak_bins_list['peak_bin']-1  # position is bin number - 1

In [8]:
pulsies_names = peak_bins_list['GRB_name']

In [9]:
# get a list of redshifts
redshifts = []
for trig, grb, z in pulsies:
    redshifts.append(z)

In [10]:
# get decay phase of fit_results

decay_results = []

for fit_result in fit_results:
    
    index = fit_results.index(fit_result)
    
    decay_result = fit_result[peak_bins_id[index]:]
    decay_results.append(decay_result)

In [11]:
len(decay_results)

38

In [12]:
sum = 0
for fit_result in fit_results:
    sum += len(fit_result)
print sum

281


In [13]:
sum = 0
for decay_result in decay_results:
    sum += len(decay_result)
print sum

166


In [ ]:
for decay_result in decay_results:
    
    index = decay_results.index(decay_result)
    GRB_name = pulsies_names[index]
    models_writer(GRB_name, decay_result)

In [15]:
# now, calculate physical properties and write all to one file

for decay_result in decay_results:
    index = decay_results.index(decay_result)
    GRB_name = pulsies_names[index]
    
    betas = get_betas(decay_result,GRB_name)
    Epeak_rest, Epeak_rest_err = Epeak_rest_frame(decay_result, GRB_name, redshifts[index])
    Flux, Flux_err, Lumin, Lumin_err = Flux_and_lumin_rest_frame(decay_result, GRB_name, redshifts[index], Epeak_rest)
    write_grb_data(GRB_name, Epeak_rest, Epeak_rest_err, Lumin, Lumin_err, betas)

In [1]:
# read the data and pop out betas
from pandas import read_csv
pandas_data = read_csv('correlation_data_f5.txt', delimiter='\t')

In [2]:
drop_list = []
for i in range(len(pandas_data.beta)):
    if(pandas_data.beta[i]>=-2):
        drop_list.append(i)

In [3]:
drop_list

[11, 31, 34, 41, 43, 45, 47, 51, 99, 135, 148, 150, 159, 162, 163]

In [4]:
len(pandas_data)

166

In [5]:
pandas_data = pandas_data.drop(drop_list)

In [6]:
len(pandas_data)

151

In [12]:
pandas_data.to_csv('/data29s/fermi/abacelj/GRB_correlations/Stan/data_for_fit_f5')